This notebook contains the investigation of the data that generated the preprocessing steps that are contained within the `process_data.py` script.

In [54]:
import pandas as pd
import numpy as np

import sqlalchemy
import os

# Raw input

## Messages

In [3]:
raw_messages = pd.read_csv('../data/disaster_messages.csv')

In [4]:
raw_messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [5]:
raw_messages.shape

(26248, 4)

## Categories

In [6]:
raw_categories = pd.read_csv('../data/disaster_categories.csv')

In [7]:
raw_categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [8]:
raw_categories.shape

(26248, 2)

Categories will be preprocessed before joining to set categories

### Fetching categories names

In [9]:
expanded_categories =  raw_categories['categories'].str.split(';', expand = True)

In [10]:
# Extracting categories

for i, value in enumerate(expanded_categories.iloc[0]):

    cat_name = value.split('-')[0]

    if i == 0:

        categories_name = [cat_name]

    else:

        categories_name.append(cat_name)

In [11]:
categories_name[0:5]

['related', 'request', 'offer', 'aid_related', 'medical_help']

In [12]:
expanded_categories.columns = categories_name

In [13]:
def get_category_label(frame, cat_cols):

    for col in frame[cat_cols].columns:

        frame[col] = frame[col].str.split('-').str[1]

        frame[col] = frame[col].astype(int)

    return frame

In [14]:
get_category_label(expanded_categories, expanded_categories.columns)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26245,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26246,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
raw_categories = pd.concat([raw_categories['id'], expanded_categories], axis = 1)

In [16]:
raw_categories.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Generating Full raw dataframe

In [17]:
df_raw = raw_messages.merge(raw_categories,
                            on = 'id',
                            how = 'left',
                            indicator=True)

In [18]:
df_raw.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,_merge
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,both
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,both
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both


In [19]:
# Are all messages in both tables?

df_raw['_merge'].value_counts()

both          26386
left_only         0
right_only        0
Name: _merge, dtype: int64

In [20]:
df_raw.drop(columns = '_merge', inplace = True)

In [21]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26386 entries, 0 to 26385
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26386 non-null  int64 
 1   message                 26386 non-null  object
 2   original                10246 non-null  object
 3   genre                   26386 non-null  object
 4   related                 26386 non-null  int32 
 5   request                 26386 non-null  int32 
 6   offer                   26386 non-null  int32 
 7   aid_related             26386 non-null  int32 
 8   medical_help            26386 non-null  int32 
 9   medical_products        26386 non-null  int32 
 10  search_and_rescue       26386 non-null  int32 
 11  security                26386 non-null  int32 
 12  military                26386 non-null  int32 
 13  child_alone             26386 non-null  int32 
 14  water                   26386 non-null  int32 
 15  fo

# Empty columns

In [22]:
# What columns are empty?
df_raw.isna().mean()[df_raw.isna().mean() != 0]

original    0.611688
dtype: float64

In [23]:
# What are empty values in the 'original' column?

df_raw[df_raw['original'].isna()][['message','original']].head(15)

,message,original
7483,NOTES: It mark as not enough information,NaN
9956,My thoughts and prayers go out to all the live...,NaN
9957,I m sorry for the poor people in Haiti tonight...,NaN
9958,RT selenagomez UNICEF has just announced an em...,NaN
9959,lilithia yes 5.2 magnitude earthquake hit mani...,NaN
9960,RT TheNewsBlotter RT caribnews On Call Interna...,NaN
9961,Most Eureka homeowners won&#39 t collect on qu...,NaN
9962,Haiti hit by largest earthquake in over 200 ye...,NaN
9963,RT pinkelephantpun Earthquake Relief Donate ht...,NaN
9964,Praying that today s earthquake in #Haiti mini...,NaN


# Duplicates

In [24]:
df_raw.duplicated().sum()

170

In [25]:
df_raw[df_raw.duplicated(keep = False)].head(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
162,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
163,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
164,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
165,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
657,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_raw.drop_duplicates(inplace = True)

# Investigating unique values in categories

In [27]:
for col in df_raw.select_dtypes(include = ['int32']):

    unique_values = df_raw[col].unique()

    # Check for columns that are not binary
    if len(unique_values) != 2:
        
        print(col)
        print(unique_values, end = '\n\n')

related
[1 0 2]

child_alone
[0]



It can be that `child_alone` might only not have a message labeled to that category.  
This is a labelling issue and will not be considered.

`related` on the other hand, should not have '2' in it. It will be further investigated.

In [28]:
df_raw[df_raw['related'] == 2].head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,146,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
221,263,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307,373,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
462,565,"Bonsoir, on est a bon repos aprs la compagnie ...",Bonswa nou nan bon repo apri teleko nan wout t...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
578,700,URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...,r et Salon Furterer. mwen se yon Cosmtologue. ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


They look like normal messages. Will be replaced with 1

In [29]:
df_raw['related'].replace(2,1, inplace = True)

# Investigating genres

In [30]:
df_raw['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

In [31]:
df_raw['genre'].value_counts(dropna = False)

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

Genre looks OK. All categories are populated.

# Messages that are not alphabetical
Maybe some messages are completely unrelated to texts

In [32]:
df_raw[df_raw['message'].str.match(r'^\W+$')]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7609,8497,//// // @:@,//// // @lt : gt@,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12293,14315,,a,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12297,14329,.,a,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12330,14428,,Shikarpur a shikarpur a,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


These look like wrong messages. Will be dropped

In [33]:
df_raw[df_raw['message'].str.match(r'^\W+$')].index

Int64Index([7609, 12293, 12297, 12330], dtype='int64')

In [34]:
df_raw.drop(df_raw[df_raw['message'].str.match(r'^\W+$')].index, inplace = True)

# Identifying not English messages

Different languages on the dataset can throw a model off.  
They will generate noise.  

Messages that don't have originals seem to be in english. This can be further investigated.  
There are messages that aren't in english on the message field. Id 804, 11200 for example

In [35]:
from langdetect import detect, DetectorFactory

In [36]:
DetectorFactory.seed = 123

In [37]:
def infer_lang(txt):

    try:

        lang = detect(txt)

    except:

        lang = None

    return lang

In [38]:
df_raw['infered_language'] = df_raw['message'].apply(infer_lang)

In [39]:
df_raw[df_raw['infered_language'].isna()]['message'].values

array(['(http://www.guardian.co.uk/global-development/2013/jan/16/somali-ngos-mogadishu-street-children)'],
      dtype=object)

Not being able to infer the language of a message that was actually a link.

In [40]:
df_raw[df_raw['infered_language'] != 'en'].shape

(406, 41)

In [41]:
df_raw[df_raw['infered_language'] != 'en'].sample(15, random_state= 1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,infered_language
12425,14643,gaav valay aahar aor aarthik samashyaa saamna ...,villagers r facing food and financial problm k...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,so
3289,3775,NOTES: personal request. Not an emergency.,Mwn vle yon ti infomasyon plis tj,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,fr
12284,14294,AURANG ZEB S/O GULAB DIN DIST MIANWALI TEH ESS...,"/ / a, a, a.",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,so
4697,5312,"Please, help Lagonave. Help, Help,Help.",MEZANMI WE SA NAFE POU LAGONAV,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,nl
6762,7619,"I am a pupil at Mombin Crochu, Nord-est depart...",MWEN MENM YON ELEV MOMBIN CROCHI DEPATMAN NODES,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ca
12248,14186,KHALIL AHMAD ARIAN KOTLA MUGHLAN TAHSEL JAMPUR...,KHALIL AHMAD ARIAN KOTLA MUGHLAN. TAHSIL JAMPU...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,de
15820,18522,(http://www.guardian.co.uk/global-development/...,NaN,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
11853,13591,faltou energia aqui mais ja voltou. to indo pr...,NaN,social,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pt
1256,1483,GEN YON TIBEBE KI MALAD NAN KOU PASKE BLOK TON...,VIRE NAN BO KKOTI YZIN NAN. WAP JWENN YON KAFO...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,de
8129,9060,f k yo ap mal viv nan ansagal lagonav.msi.,f ke yo ap mal viv nan ansagale lagonav. mesi.,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,tl


The inferece looks like good enough. There are naturally mistakes, but dropping these rows might benefit more than do harm.

# Text cleaning

## Identifying links

In [42]:
df_raw[df_raw['message'].str.contains(r'http(s)?:\/\/[\w.-_]+\s', regex = True)]

C:\Users\Lucca\AppData\Local\Temp\ipykernel_17104\1027339002.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_raw[df_raw['message'].str.contains(r'http(s)?:\/\/[\w.-_]+\s', regex = True)]


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,infered_language
5061,5737,If you want to find a Job at an NGO or the Gov...,4636 : Si-w vle jwenn job nan yon ONG osinon g...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
7403,8283,http://wap.sina.comhttp://wap.sina.com,http : //wap. sina. comhttp : //wap. sina. com,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
8913,9947,Nokia.com http://ea.mobile.nokia.com/ea/graphics,Nokia.com http://ea.mobile.nokia.com/ea/graphics,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pt
9788,10960,BEGIN:VBKM VERSION:1.0 TITLE:Digicel Live Ha U...,BEGIN:VBKM VERSION:1.0 TITLE:Digicel Live Ha U...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
11470,13114,RT stro_Soichi: http://twitpic.com/15wu5u - Sa...,NaN,social,1,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19592,22704,"Dengue, an acute febrile disease spread by the...",NaN,news,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,en
21924,25267,The UN Capital Development Fund (UNCDF) [ http...,NaN,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
22037,25390,"In a November overview, the UN Children's Fund...",NaN,news,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,en
22659,26072,The map [ http://pakrelief.crowdmap.com ] on t...,NaN,news,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,en


In [43]:
df_raw['message_cleaned'] = df_raw['message'].str.replace(r'http(s)?:\/\/[\w.-_]+\s', 'url_placeholder ', regex = True)

## Removing non alphabetical characters

In [44]:
pat_not_alnum = r'[^\w\s]'

In [45]:
df_raw[df_raw['message'].str.contains(pat_not_alnum, regex = True)]['message'].head()

0    Weather update - a cold front from Cuba that c...
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
5               Information about the National Palace-
7    Please, we need tents and water. We are in Sil...
Name: message, dtype: object

In [46]:
df_raw['message_cleaned'] = df_raw['message_cleaned'].str.replace(pat_not_alnum,' ',regex = True)

## Normalizing case

In [47]:
df_raw['message_cleaned'] = df_raw['message_cleaned'].str.lower()

In [48]:
df_raw.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,infered_language,message_cleaned
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,en,weather update a cold front from cuba that c...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,1,0,1,0,0,0,0,0,en,is the hurricane over or is it not over
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,no,looking for someone but no name
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,en,un reports leogane 80 90 destroyed only hospi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,en,says west side of haiti rest of the country ...


## Removing double spacing

In [49]:
remove_double_spacing = lambda x: ' '.join(x.split())

In [51]:
df_raw['message_cleaned'] = df_raw['message_cleaned'].apply(remove_double_spacing)

df_raw['message_cleaned'] = df_raw['message_cleaned'].str.strip()

In [52]:
df_raw.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,infered_language,message_cleaned
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,en,weather update a cold front from cuba that cou...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,1,0,1,0,0,0,0,0,en,is the hurricane over or is it not over
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,no,looking for someone but no name
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,en,un reports leogane 80 90 destroyed only hospit...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,en,says west side of haiti rest of the country to...


# Export to SQL

In [69]:
conn = sqlalchemy.create_engine('sqlite:///' + os.path.abspath('../data/DisasterResponse.db'))

In [70]:
df_raw.to_sql('ANALYTICAL_TABLE', conn)

26212